### week two exercise

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import os
import json
import base64
from io import BytesIO
import sqlite3
from PIL import Image
import gradio as gr

In [ ]:
# Initialization

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_BASE_URL = 'http://localhost:11434/v1'
DB = "prices.db"

api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()

In [ ]:
question = """
Hi there, I'm an embedded software engineer and I'm feeling kinda bored.
Tell me an interesting fact about freertos and exciting projects I can build with it!
I'd be less bored if you could tell me how to build those fun projects!
"""

In [ ]:
system_prompt = """you are a helpful assistant that can give a bored embedded software engineer project ideas
                   in a fun way and how to implement them. First, have a conversation with the user and only suggest
                   a project when they ask you to.
                   When you suggest a project to the user, you must call the suggest_project tool with the project
                   name and a short description so an image of the project can be generated for them in the same response.
                   give detailed project ideas in a technical way that such an engineer can implement. also, tell
                   the user how much the project would cost!
                   Give the user a breakdown of the cost. If you don't know the price of a component, 
                   say you don't know and don't guess or hallucinate the price."""


In [ ]:

import sqlite3

DB = 'components.db'

components_data = [
    ('ESP32-WROOM-32D Development Board', 8.50),
    ('Arduino Nano (Clone)', 4.25),
    ('Raspberry Pi Pico', 4.00),
    ('DHT22 Temperature & Humidity Sensor', 3.75),
    ('HC-SR04 Ultrasonic Distance Sensor', 2.10),
    ('SG90 Micro Servo Motor', 2.50),
    ('16x2 LCD Display with I2C Adapter', 5.50),
    ('TP4056 Li-Po Charging Module', 0.85),
    ('NRF24L01+ Wireless Transceiver', 1.20),
    ('Breadboard (830 Points)', 4.00),
    ('Jumper Wire M-M (40pcs)', 1.50),
    ('10k Ohm Potentiometer', 0.60),
    ('SSD1306 0.96 inch OLED Display', 4.50),
    ('BME280 Pressure/Temp/Hum Sensor', 9.00),
    ('MPU-6050 6-Axis Gyro/Accel', 3.25),
    ('Resistor Kit (600 pcs, Assorted)', 12.00),
    ('Ceramic Capacitor Kit (300 pcs)', 8.50),
    ('Logic Level Converter (4-channel)', 1.10),
    ('Active Buzzer 5V', 0.45),
    ('WS2812B RGB LED Strip (1m/60 LEDs)', 11.00),
    ('Micro SD Card Module', 1.80),
    ('PIR Motion Sensor (HC-SR501)', 1.95),
    ('Shift Register 74HC595', 0.35),
    ('2N2222 NPN Transistor (10pk)', 1.20),
    ('DS3231 RTC Module', 3.80)
]

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS components (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            component TEXT NOT NULL,
            price REAL NOT NULL
        )
    """)
    
    cursor.execute('SELECT COUNT(*) FROM components')
    count = cursor.fetchone()[0]
    if count == 0:
        cursor.executemany(
            'INSERT INTO components (component, price) VALUES (?, ?)',
            components_data
        )
    conn.commit()

def get_component_price(component_name):
    print(f"DATABASE TOOL CALLED: Getting price for component '{component_name}'", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM components WHERE component = ?', (component_name,))
        result = cursor.fetchone()
        return f"Price of '{component_name}' is ${result[0]}" if result else f"No price data available for component '{component_name}'"

In [ ]:
component_price_function = {
    "name": "get_component_price",
    "description": "Get the price of the specified electronics component by name.",
    "parameters": {
        "type": "object",
        "properties": {
            "component_name": {
                "type": "string",
                "description": "The name of the electronics component the user wants the price for",
            },
        },
        "required": ["component_name"],
        "additionalProperties": False
    }
}

suggest_project_function = {
    "name": "suggest_project",
    "description": "Call this when you are suggesting a project to the user. Use it as soon as you suggest a project so an image of the project can be shown in the same response.",
    "parameters": {
        "type": "object",
        "properties": {
            "project_name": {
                "type": "string",
                "description": "Short name of the project you are suggesting (e.g. 'Smart Plant Watering System')",
            },
            "short_description": {
                "type": "string",
                "description": "Brief description of the project for image generation (e.g. 'ESP32-based system with soil sensor and pump')",
            },
        },
        "required": ["project_name", "short_description"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": component_price_function}, {"type": "function", "function": suggest_project_function}]

In [ ]:
def artist(component_name):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"A vibrant pop-art illustration highlighting the electronics \
            component '{component_name}' and its common uses in technology or gadgets. \
                Make it visually appealing and professional.",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))


def artist_project(project_name, short_description):
    """Generate an image for a suggested project (called when the LLM suggests a project)."""
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"A vibrant, inspiring illustration of an embedded/electronics project: '{project_name}'. "
               f"{short_description}. Show the concept in a clear, technical yet appealing way.",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Try replacing "onyx" with "alloy" or "coral" for different voices
        input=message
    )
    return response.content

In [ ]:
def handle_tool_calls_and_return_components(message):
    responses = []
    components = []
    suggested_project = None 
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_component_price":
            arguments = json.loads(tool_call.function.arguments)
            component = arguments.get('component_name')
            components.append(component)
            price_details = get_component_price(component)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "suggest_project":
            arguments = json.loads(tool_call.function.arguments)
            suggested_project = {
                "project_name": arguments.get("project_name", "Project"),
                "short_description": arguments.get("short_description", ""),
            }
            responses.append({
                "role": "tool",
                "content": "Project suggestion recorded; image will be generated for the user.",
                "tool_call_id": tool_call.id
            })
    return responses, components, suggested_project

In [ ]:
def chat(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)
    components = []
    suggested_project = None
    image = None

    # Handle tool calls (get_component_price, suggest_project) as long as they're requested
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses, components, this_suggested = handle_tool_calls_and_return_components(message)
        if this_suggested:
            suggested_project = this_suggested  # keep project suggestion from this turn
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)

    reply = response.choices[0].message.content or ""
    history += [{"role": "assistant", "content": reply}]

    voice = talker(reply) if reply else None

    # Generate image when the LLM suggested a project (same response as the suggestion)
    if suggested_project:
        image = artist_project(
            suggested_project["project_name"],
            suggested_project["short_description"]
        )
    elif components:
        # Fallback: component image if only price lookup happened
        image = artist(components[0])

    return history, voice, image


In [ ]:
# Callback to post a user message into history for the chatbot
def put_message_in_chatbot(message, history):
    # Reset input and append user message to chat history
    return "", history + [{"role": "user", "content": message}]

# Gradio UI definition and event wiring
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

    # Connect textbox submit to chat update and OpenAI + image + voice pipeline
    message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

# Launch the UI in the browser with simple authentication
ui.launch(inbrowser=True, auth=("denis", "hello"))